**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Tue Jun 16 13:49:05 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
'Colab Notebooks'   YOLO   yolov3


**1) Clone the Darknet**



In [3]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 13714, done.
remote: Total 13714 (delta 0), reused 0 (delta 0), pack-reused 13714
Receiving objects: 100% (13714/13714), 12.31 MiB | 16.61 MiB/s, done.
Resolving deltas: 100% (9353/9353), done.


**2) Compile Darknet using Nvidia GPU**


In [4]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:910:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1391:14: warning: unused variable ‘buff’ [-Wunused-variable]
         char buff[100];
              ^~~~
./src/image_opencv.cpp:1367:9: warning: unused variabl

**3) Configure Darknet network for training YOLO V3**

In [0]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [0]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [7]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [0]:
!echo "Koala" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [9]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-06-16 13:53:34--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M   118KB/s    in 16m 45s 

2020-06-16 14:10:20 (158 KB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [10]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/000e1654f1d9b2cb.txt  
  inflating: data/obj/00baa39512651684.jpg  
  inflating: data/obj/00baa39512651684.txt  
  inflating: data/obj/00ff1bf898beb6ea.jpg  
  inflating: data/obj/00ff1bf898beb6ea.txt  
  inflating: data/obj/00ff63ca740b0723.jpg  
  inflating: data/obj/00ff63ca740b0723.txt  
  inflating: data/obj/0a76d04791eb3eb4.jpg  
  inflating: data/obj/0a76d04791eb3eb4.txt  
  inflating: data/obj/0ad73327a7b426b4.jpg  
  inflating: data/obj/0ad73327a7b426b4.txt  
  inflating: data/obj/0b3afdbac018dd7f.jpg  
  inflating: data/obj/0b3afdbac018dd7f.txt  
  inflating: data/obj/0b5599a0ea6df06b.jpg  
  inflating: data/obj/0b5599a0ea6df06b.txt  
  inflating: data/obj/0bc29fadc0e384f6.jpg  
  inflating: data/obj/0bc29fadc0e384f6.txt  
  inflating: data/obj/0be08444b76cde10.jpg  
  inflating: data/obj/0be08444b76cde10.txt  
  inflating: data/obj/0c5f6d4940cba28b.jpg  
  inflating: data/obj/0c5f6d4940cba28b.txt  
  inflating: data/

In [11]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.3490229375 0.2693307516339869 0.20312499999999997 0.16317900000000002

['0', '0.3490229375', '0.2693307516339869', '0.20312499999999997', '0.16317900000000002']
0 data/obj/05942b9c335c17e5.txt
0 0.3490229375 0.2693307516339869 0.20312499999999997 0.16317900000000002
0 0.1802734375 0.49036491666666665 0.24843800000000002 0.24999999999999992

['0', '0.1802734375', '0.49036491666666665', '0.24843800000000002', '0.24999999999999992']
1 data/obj/0b5599a0ea6df06b.txt
0 0.1802734375 0.49036491666666665 0.24843800000000002 0.24999999999999992
0 0.141797375 0.305208 0.16874899999999998 0.15833400000000006

['0', '0.141797375', '0.305208', '0.16874899999999998', '0.15833400000000006']
1 data/obj/0b5599a0ea6df06b.txt
0 0.141797375 0.305208 0.16874899999999998 0.15833400000000006
0 0.3333979375 0.26875 0.23281300000000002 0.25

['0', '0.3333979375', '0.26875', '0.23281300000000002', '0.25']
1 data/obj/0b5599a0ea6df06b.txt
0 0.3333979375 0.26875 0.23281300000000002 0.25
0 0.36484375 0.685937166

In [12]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/00ff63ca740b0723.jpg', 'data/obj/21002233b8c97856.jpg', 'data/obj/e9c08b1c69dc69c4.jpg', 'data/obj/7192d975311f2444.jpg', 'data/obj/73bb89f2a9c8c480.jpg', 'data/obj/dd2ad04287cae465.jpg', 'data/obj/e01b12c5ef2aa38d.jpg', 'data/obj/1115ab238c59f80e.jpg', 'data/obj/a34fb6af4b0ef47f.jpg', 'data/obj/53a189a62f972873.jpg', 'data/obj/2ddb5fed1a6a1419.jpg', 'data/obj/1ce9d33a8e76f0fb.jpg', 'data/obj/9b51461e66181426.jpg', 'data/obj/6642d011c33f5eac.jpg', 'data/obj/0d83df9b67b7e8cf.jpg', 'data/obj/644b0c7f2afbdbe6.jpg', 'data/obj/30cba558171da1ef.jpg', 'data/obj/5743b6bc1a197836.jpg', 'data/obj/adce5a677a041370.jpg', 'data/obj/7421b8827efdfc7c.jpg', 'data/obj/24f026e14ab33414.jpg', 'data/obj/e36f3cab13c388f6.jpg', 'data/obj/7e333cbd8b012c94.jpg', 'data/obj/d01cd12df52e6791.jpg', 'data/obj/e5d0987c2bbef4f1.jpg', 'data/obj/0e7b156c3e2ff618.jpg', 'data/obj/017d69b772ee34a7.jpg', 'data/obj/764a7fbe9cc710f0.jpg', 'data/obj/34ef2028aa6cace3.jpg', 'data/obj/158906859693fb9e.jpg', 'data/obj

In [0]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [0]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
yolov3_training
 0 : compute_capability = 370, cudnn_half = 0, GPU: Tesla K80 
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Shortcut Layer: 5,  wt = 0, wn = 0, outputs: 104 x

In [0]:
imShow('chart.png')